<a href="https://colab.research.google.com/github/maybachar/getpet-recognition/blob/master/train_model_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.models import model_from_json

#append to lables list the suitale lable for each image using info from csv file
#and add the number of breed instead the name breed(using dictionary).   
def create_lables_list(image_id, lables):
    image_id = image_id.split("/")[-1].split(".")[0]
    # get breed name by image id from labels.csv
    breed_name = list(labels_file[labels_file.id == image_id]["breed"])[0]
    # get the number of this breed and append it to list.
    breed_idx = breed_num_dic[breed_name]
    lables.append(breed_idx)


def build(size, num_classes):
    # the input size the model will get. default size for mobile and 3 is for colors.
    inputs = Input((size, size, 3))
    # create the base pre-trained model.
    base_model = MobileNetV2(input_tensor=inputs, include_top=False,
                           weights="imagenet")
    # set all parameters to be trainable
    base_model.trainable = True
    # get the model so we can add layers to it.
    x = base_model.output
    # add a global spatial average pooling layer
    x = GlobalAveragePooling2D()(x)
    # include a dropout layer to minimize the overfitting.
    x = Dropout(0.2)(x)
    # using activation function RELU- widely used in CNN.
    x = Dense(1024, activation="relu")(x)
    # add softmax layer for getting probabilities on the breeds.
    x = Dense(num_classes, activation="softmax")(x)
    # create the model with it's layers.
    model = tf.keras.Model(inputs, x)
    return model


def fix_data_element(path, y):
    num_class = 12
    size = 224
    path = path.decode()
    # loads a color image from the specified file.
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    # change the width, height of an image.
    image = cv2.resize(image, (size, size))
    # normaliztion.
    image = image / 255.0
    image = image.astype(np.float32)
    # one hot encoding to lable (all zero except for the place y).
    label = [0] * num_class
    label[y] = 1
    label = np.array(label)
    label = label.astype(np.int32)
    return image, label

def parse(x, y):
    num_class = 12
    size = 224
    # wrap numpy function as an operation in TensorFlow function.
    x, y = tf.numpy_function(fix_data_element, [x, y], [tf.float32, tf.int32])
    # resize image to the image size mobilenet expect to get.
    x.set_shape((size, size, 3))
    y.set_shape((num_class))
    return x, y

def fix_dataset(x, y, batch=8):
    # get the slices of x and y into one dataset.
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    # execute fix_data function on every element of the Dataset separately.
    dataset = dataset.map(parse)
    # combines consecutive elements of a dataset object into batches.
    dataset = dataset.batch(batch)
    return dataset



if __name__ == "__main__":
    # image size
    size = 224
    num_breeds = 12
    # learning rate 0.0001.
    lr = 0.0001
    batch = 16
    epochs = 5

    # paths of train data folder and csv file with labels of each pic.
    path = "/content/drive/MyDrive/cat-breed-identification/"
    train_path = os.path.join(path, "images/images/*")
    labels_path = os.path.join(path, "cat_labels.csv")
    # import the CSV file.
    labels_file = pd.read_csv(labels_path)
    # get all unique breeds in file.
    breed = labels_file["breed"].unique()
    # create dictionary of breed and number.
    breed_num_dic = {name: i for i, name in enumerate(breed)}
    # ids of photos
    ids = glob(train_path)
    # list of breed index by order of image id
    lables = []
    # adding the num of breed-the lable of each image to lables list.
    for image_id in ids:
        create_lables_list(image_id, lables)

    # work only with N image id and N lables of the breed id which suitable to the images.
    ids = ids[:2350]
    lables = lables[:2350]

    # spliting data to train and validation.
    train_x, validation_x = train_test_split(ids, test_size=0.2, random_state=42)
    train_y, validation_y = train_test_split(lables, test_size=0.2, random_state=42)
    #create a CNN model with it's layers.
    model = build(size, num_breeds)
    #Configures the model for training. optimizer- adam.
    model.compile(loss="categorical_crossentropy", optimizer=Adam(lr), metrics=["acc"])
    # dataset
    trainSet = fix_dataset(train_x, train_y, batch=batch)
    validationSet = fix_dataset(validation_x, validation_y, batch=batch)

    # train
    #set of functions to be applied at training procedure:
    # modelCheckPoint saves the model after every epoch.
    # Reduce learning rate when a metric has stopped improving.
    callbacks = [
        ModelCheckpoint("model.h5", verbose=1, save_best_only=True),
        ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.000001)
    ]
    # Trains the model for number of epochs. evaluate the loss at the end of each epoch.
    model.fit(trainSet, validation_data=validationSet, epochs=epochs, callbacks=callbacks)
    # save the model and it's weights in H5 format to jeson file.
    model_json = model.to_json()
    with open("/content/drive/MyDrive/cat-breed-identification/model.json", "w") as json_file:
        json_file.write(model_json)
        model.save_weights("/content/drive/MyDrive/cat-breed-identification/model.h5")
        print("Saved model to disk")



9412608/9406464 [==============================] - 0s 0us/step
Epoch 1/5
118/118 [==============================] - 1560s 13s/step - loss: 1.1061 - acc: 0.6457 - val_loss: 0.9818 - val_acc: 0.6596

Epoch 00001: val_loss improved from inf to 0.98184, saving model to model.h5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/5
118/118 [==============================] - 357s 3s/step - loss: 0.2070 - acc: 0.9436 - val_loss: 0.7029 - val_acc: 0.7362

Epoch 00002: val_loss improved from 0.98184 to 0.70285, saving model to model.h5
Epoch 3/5
118/118 [==============================] - 356s 3s/step - loss: 0.0456 - acc: 0.9957 - val_loss: 0.5695 - val_acc: 0.8021

Epoch 00003: val_loss improved from 0.70285 to 0.56948, saving model to model.h5
Epoch 4/5
118/118 [==============================] - 354s 3s/step - loss: 0.0133 - acc: 0.9995 - val_loss: 0.6049 - val_acc: 0.7957

Epoch 00004: val_loss did not improve from 0.56948
Epoch 5/5
118/118 [==============================] - 362s 3s/step - loss: 0.0068 - acc: 1.0000 - val_loss: 0.5254 - val_acc: 0.8277

Epoch 00005: val_loss improved from 0.56948 to 0.52538, saving model to model.h5
Saved model to disk
